# $(3,11)$--Matriods

In this notebook, we provide the computations used in the proof of Proposition 4.4, which asserts that $\operatorname{Gr}(\mathsf{Q};\mathbb{C})$ is smooth for all $\mathbb{C}$--realizable $(3,11)$--matroids. 

In [1]:
using Oscar

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.12.1 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2023 by The OSCAR Development Team


In [2]:
cd("..")

In [3]:
currentDir = pwd() # make sure you are running from the main directory.
include(joinpath(currentDir, "src/fileHandling.jl"))
include(joinpath(currentDir, "src/isolate3Lines.jl"))
include(joinpath(currentDir, "src/matroid_realization.jl"))
include(joinpath(currentDir, "src/Jacobian_Criterion.jl"))

realization_space_2_singular_locus (generic function with 1 method)

First we isolate those that satisfy the three lines property, i.e., every element of the ground set is contained in at least 3 lines (cyclic flats of rank 2). There are **16234** such matroids.

**Warning** this cell takes a long time to run. 

In [ ]:
db = Polymake.Polydb.get_db();
collection = db["Matroids.Small"];
d3n11 = Polymake.Polydb.find(collection, Dict("RANK" => 3, "SIMPLE"=>true, "N_ELEMENTS"=>11));

n3C11 = subsets(collect(1:11), 3);
n3C11 = sort(n3C11, by =  x-> reverse(x));

lines_3 = []
for c in d3n11
    Q = Matroid(c)
    ns = count_3_lines_thru_all_points(Q)
    if length(ns) == 0
        continue
    end    
    if minimum(ns) >= 3
        push!(lines_3, to_revlex(Q, n3C11))
    end
end

This data is precomputed in `d3n11/data/3lines_3_11.dat`.

In [4]:
lines_3_precomputed = vec(readlines(joinpath(currentDir, "d3n11/data/3lines_3_11.dat")));
length(lines_3_precomputed)

16234

In [ ]:
Set(lines_3) == Set(lines_3_precomputed)

Alternately, run `julia d3n11/src/isolate_3_lines.jl`.

The file `d3n11/data/nonrealizable_3_11.dat` contains those matroids in `d3n11/data/3lines_3_11.dat` that are not realizable over $\mathbb{Q}$. There are **4718** such matroids. 

In [5]:
not_realizable = vec(readlines(joinpath(currentDir, "d3n11/data/nonrealizable_3_11.dat")));
length(not_realizable)

4718

**Warning:** the cell below takes ~10 minutes to complete. 

In [ ]:
test_nonrealizable = []

for Qstr in not_realizable
    Q = matroid_from_revlex_basis_encoding(Qstr, 3, 11) 
    As = rank_plus1_circuits(Q) 
    MR = new_matroid_realization_space(Q, As[1]; F=QQ, saturate=true)
    push!(test_nonrealizable, !MR.representable)
end

In [ ]:
all(test_nonrealizable)

Alternately, run `julia d3n11/src/test_nonrealizable.jl`.

The files `d3n11/data/zero_ideal_3_11.dat`, `d3n11/data/univariate_ideal_3_11.dat`, `d3n11/data/multivariate_principal_ideal_3_11.dat`, record matroids and a maximal circuit. A pair of matroid and circuit is recorded as a `*/0` vector. The first 11 entries record the circuit (e.g., `**0*00*0000` is the circuit `[1,2,4,7]`) and the rest record the matroid in the usual revlex basis encoding. 

**Note:** One could check that these are smooth using the `singular_locus` function in `src/Jacobian_Criterion.jl`. To make this computation more transparent, we carry out the smoothness check explicitly.

The files `d3n11/data/zero_ideal_3_11.dat` contains those matroids in `d3n11/data/3lines_3_11.dat` and a maximal circuit such that the ideal of the realization space is zero. There are **2220** such matroids. Therefore these realization spaces are smooth. 

In [6]:
zero_ideal = vec(readlines(joinpath(currentDir, "d3n11/data/zero_ideal_3_11.dat")));
length(zero_ideal)

2220

**Warning:** this cell takes (fill in, something like 15 min) minutes to run.

In [ ]:
test_zero = []
for cir_Qstr in zero_ideal
    A = [i for i in 1:11 if string(cir_Qstr[i]) == "*"]     
    Qstr = cir_Qstr[12:176]
    Q = matroid_from_revlex_basis_encoding(Qstr, 3, 11) 
    MR = new_matroid_realization_space(Q, A; F=QQ, saturate=false)
        
    MR = reduce_ideal_full(MR)
    I = MR.defining_ideal
    push!(test_zero, iszero(I))
end

In [ ]:
all(test_zero)

Alternately, run `julia d3n11/src/test_zero_ideal.jl`.

The file `d3n11/data/univariate_ideal_3_11.dat` contains those matroids in `d3n11/data/3lines_3_11.dat` and a maximal circuit such that the ideal of the realization space is univariate. There are **3658** such matroids. These realization spaces are smooth provided the discriminant is nonzero. 

In [7]:
univariate_ideal = vec(readlines(joinpath(currentDir, "d3n11/data/univariate_ideal_3_11.dat")));
length(univariate_ideal)

3658

If $I\subset \mathbb{C}[x_1,\ldots,x_k]$ is an ideal principally generated by a polynomial in a single variable $x_1$, then $I$ defines a smooth scheme if and only if $f$  and $df/dx$ are relatively prime.  

**Warning:** this cell takes (fill in, something like 15 min) minutes to run.

In [ ]:
test_univariate = []
for cir_Qstr in univariate_ideal
    A = [i for i in 1:11 if string(cir_Qstr[i]) == "*"]     
    Qstr = cir_Qstr[12:176]
    Q = matroid_from_revlex_basis_encoding(Qstr, 3, 11) 
    MR = new_matroid_realization_space(Q, A; F=QQ, saturate=true)
        
    MR = reduce_ideal_full(MR)
    I = MR.defining_ideal
    vs = ideal_vars(gens(I))
    length(vs) != 1 && error("not univariate")
    f = gens(I)[1]; #println("f = ", f); 
    df = derivative(f, vs[1]); #println("df = ", df); 
    push!(test_univariate, isone(ideal(MR.ambient_ring, [f,df])))
end

In [ ]:
all(test_univariate)

Alternately, run `julia d3n11/src/test_univariate_ideal.jl`.

The file `d3n11/data/multivariate_principal_ideal_3_11.dat` contains those matroids in `d3n11/data/3lines_3_11.dat` and a maximal circuit such that the defining ideal of the realization space is multivariate and principal. There are **5638** such matroids.

In [9]:
multivariate_principal = vec(readlines(joinpath(currentDir, "d3n11/data/multivariate_principal_ideal_3_11.dat")));
length(multivariate_principal)

5638

By the Jacobian criterion, the affine scheme $\mathsf{Spec}(S^{-1}\mathbb{Q}[x_1,\ldots,x_n] / \langle f \rangle)$ is smooth if and only if the ideal

$$
J = \langle f, \frac{\partial f}{\partial x_1}, \ldots, \frac{\partial f}{\partial x_n}\rangle
$$

is the unit ideal in $S^{-1} \mathbb{Q}[x_1,\ldots,x_n]$. This is equivalent to the saturation $(J:S^{\infty})$ being the unit ideal in $\mathbb{Q}[x_1,\ldots,x_n]$. We perform this explicit check here. 

**Warning:** This cell takes a long time to run.

In [ ]:
test_principal = []
for cir_Qstr in multivariate_principal
    A = [i for i in 1:11 if string(cir_Qstr[i]) == "*"]     
    Qstr = cir_Qstr[12:176]
    Q = matroid_from_revlex_basis_encoding(Qstr, 3, 11) 
    MR = new_matroid_realization_space(Q, A; F=QQ, saturate=true)
        
    MR = reduce_ideal_full(MR)
    R = MR.ambient_ring
    x = gens(R)
    I = MR.defining_ideal
    Igens = gens(I)
    length(Igens) != 1 && error("not principal") 
    JM = jacobian_matrix(R, x, Igens)
    nr, nc = size(JM) 
    J = I + ideal(R, [JM[1,c] for c in 1:nc])
    Sing = stepwise_saturation(J, MR.inequations)
    #Sing = realization_space_2_singular_locus(MR)
    push!(test_principal, isone(Sing))
end

Alternately, run `julia d3n11/src/test_principal.jl`.

Finally, we verify that all matroids have been checked. 

In [ ]:
checked_realizable = vcat(zero_ideal, univariate_ideal, multivariate_principal)
checked_realizable = [x[12:176] for x in checked_realizable]
unique!(checked_realizable)
checked = vcat(not_realizable, checked_realizable)
Set(checked) == Set(lines_3_precomputed)